In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [4]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("data/en_ewt-up-test.conllu", "test.json")

In [5]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'train.json', 'test': 'test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'predicate_position', 'predicate', 'arguments'],
        num_rows: 40482
    })
    test: Dataset({
        features: ['word', 'predicate_position', 'predicate', 'arguments'],
        num_rows: 4799
    })
})

In [6]:
datasets["train"][0]

{'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.'],
 'predicate_position': 7,
 'predicate': 'kill.01',
 'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_']}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,word,predicate_position,predicate,arguments
0,"[What, are, we, going, to, get, her, ?]",2,be.03,"[_, V, _, _, _, _, _, _]"
1,"[Excellent, customer, service, and, quality, work, .]",3,serve.02,"[ARGM-ADJ, ARG2, V, _, _, _, _]"
2,"[The, invoice, is, not, detailed, ,, so, it, is, difficult, to, see, what, you, are, paying, for, .]",3,be.01,"[_, ARG1, V, ARGM-NEG, ARG2, _, _, _, _, _, _, _, _, _, _, _, _, _]"
3,"[Question, Mark]",2,mark.02,"[ARG1, V]"
4,"[The, crop, dusters, were, to, be, part, of, a, "", second, wave, ., ""]",6,be.01,"[_, _, ARG1, _, _, V, ARG2, _, _, _, _, _, _, _]"
5,"[With, regard, to, a, global, message, ,, I, think, that, one, of, the, key, points, around, fundamental, operating, standards, are, that, they, are, intended, to, be, global, in, nature, ,, applied, to, every, commodity, and, every, location, where, we, engage, in, trading, activities, .]",24,intend.01,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG1, _, V, _, _, C-ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]"
6,"[The, early, days, were, the, days, of, competition, between, Paul, and, his, wife, and, John, and, his, wife, and, he, knew, he, had, to, compete, with, ugly, John, for, the, leadership, of, the, band, in, front, of, their, wives, and, having, Yoko, Ono, made, him, aware, of, his, lack, of, virility, and, repressed, homosexuality, ,, he, grew, the, beard, that, we, see, in, the, "", Let, it, Be, "", film, and, started, to, show, pictures, of, naked, women, in, the, same, film, ,, doing, every, effort, he, could, to, be, seen, as, a, man, ....]",90,be.03,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _, _, _, _]"
7,"[These, alleged, members, of, the, Arakan, Army, ,, the, military, wing, of, the, National, Unity, Party, in, Myanmar, ,, have, been, charged, with, illegal, entry, .]",21,be.03,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _, _, _, _]"
8,"[Tourists, like, the, other, reviewer, might, not, appreciate, their, efficiency, or, quality, ,, but, I, certainly, do, .]",8,appreciate.02,"[ARG0, _, _, _, _, ARGM-MOD, ARGM-NEG, V, _, ARG1, _, _, _, _, _, _, _, _]"
9,"[They, would, know, .]",3,know.01,"[ARG0, ARGM-MOD, V, _]"


In [9]:
datasets["train"][0]

{'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.'],
 'predicate_position': 7,
 'predicate': 'kill.01',
 'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_']}

### Tokenization

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = '[PAD]'
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])
    tokenized_input['labels'] = aligned_labels
    
    return tokenized_input


In [13]:
print(len(datasets['train']))
training_set = [tokenize(instance) for instance in datasets['train']]
test_set = [tokenize(instance) for instance in datasets['test']]


40482


In [14]:
sample = training_set[0]
print(sample)

{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': ['[PAD]', '_', '_', '_', '_', '_', '_', 'ARG0', 'V', 'ARG1', 'ARG1', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'ARGM-LOC', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']}


In [15]:
from datasets import Dataset, Features, Sequence, Value

features = Features({
    "input_ids": Sequence(feature=Value(dtype='int64')),
    "attention_mask": Sequence(feature=Value(dtype='int64')),
    "labels": Sequence(feature=Value(dtype='int64')),
})

In [16]:
unseen = ['R-ARGM-ADJ', '_', 'ARGM-ADJ']
label_list = list(set(label for instance in training_set for label in instance['labels'] + unseen))
label_dict = {label:int(i) for i, label in enumerate(list(label_list))}
label_dict['[PAD]'] = -100
for instance in training_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]
for instance in test_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]

In [17]:
dataset_train = Dataset.from_dict({"input_ids": [item['input_ids'] for item in training_set],
                             "attention_mask": [item['attention_mask'] for item in training_set],
                             "labels": [item['labels'] for item in training_set]}, features=features)

dataset_test = Dataset.from_dict({"input_ids": [item['input_ids'] for item in test_set],
                             "attention_mask": [item['attention_mask'] for item in test_set],
                             "labels": [item['labels'] for item in test_set]}, features=features)

In [18]:
#print(tokenized_input.word_ids())
print(dataset_train[0])

{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 52, 52, 52, 52, 52, 52, 0, 48, 41, 41, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 7, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, -100, -100, -100, -100, -100]}


In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
dataset_train[0]

{'input_ids': [101,
  2632,
  1011,
  23564,
  2386,
  1024,
  2137,
  2749,
  2730,
  21146,
  28209,
  14093,
  2632,
  1011,
  2019,
  2072,
  1010,
  1996,
  14512,
  2012,
  1996,
  8806,
  1999,
  1996,
  2237,
  1997,
  1053,
  4886,
  2213,
  1010,
  2379,
  1996,
  9042,
  3675,
  1012,
  102,
  3102,
  1012,
  5890,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  52,
  52,
  52,
  52,
  52,
  52,
  0,
  48,
  41,
  41,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  7,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  -100,
  -100,
  -100,
  -100,
  -100]}

In [21]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [23]:
metric = load_metric("seqeval")
metric = load_metric("seqeval")


C:\Users\katko\AppData\Local\Temp\ipykernel_32160\1988684555.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load

In [24]:
labels = list(label_list)
metric.compute(predictions=[labels], references=[labels])

C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARGM-DIS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-MNR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C-ARGM-MNR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-TMP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-pac

{'ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1-DSP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG4': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CAU': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-COM': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CXN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-DIR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-EXT': {'precisi

In [25]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
stop

NameError: name 'stop' is not defined

In [32]:
import pickle
def load_model(pickles):
    '''
    Input: string name with .pickle extension. Must be present in working directory. Check with 'pwd' command
    Loads pretrained model variable from pickle file at filepath(pickle).
    '''
    with open(pickles, 'rb') as p:
        return pickle.load(p)

In [33]:
model = load_model('basic_bert.pickle')

In [82]:
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = Trainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [75]:
x, y = dataset_test.select_columns(['input_ids','attention_mask']).with_format('torch'), dataset_test.select_columns(['labels']).with_format('torch')

In [98]:
import torch
model.eval()
with torch.no_grad():
    outputs = trainer.predict(x)


In [99]:
predictions = np.argmax(outputs.predictions, axis=1)

predictions = predictions.tolist()

In [100]:
df_test = pd.DataFrame()
df_test['predictions'] = predictions
df_test['true_labels'] = y
df_test.to_csv('outputs/test_set_predictions.csv', index=False)

In [ ]:
import pickle
def picklify(pickles,not_so_pickles):
    '''
    Saves a model or variable to a pickle file 
    '''
    with open(pickles, 'wb') as p:
        pickle.dump(not_so_pickles, p)
picklify('basic_bert.pickle',model)

In [101]:
df_test

,predictions,true_labels
0,"[8, 4, 4, 8, 34, 4, 8, 8, 5, 3, 9, 1, 4, 11, 4...","{'labels': [tensor(-100), tensor(52), tensor(5..."
1,"[25, 4, 4, 4, 2, 4, 25, 4, 4, 17, 25, 1, 4, 1,...","{'labels': [tensor(-100), tensor(52), tensor(5..."
2,"[32, 20, 6, 20, 26, 6, 31, 6, 6, 31, 31, 11, 4...","{'labels': [tensor(-100), tensor(52), tensor(5..."
3,"[19, 20, 20, 20, 26, 20, 25, 20, 20, 21, 25, 1...","{'labels': [tensor(-100), tensor(52), tensor(5..."
4,"[19, 20, 20, 20, 26, 20, 25, 20, 20, 21, 25, 1...","{'labels': [tensor(-100), tensor(52), tensor(5..."
...,...,...
4794,"[17, 14, 14, 14, 14, 14, 17, 14, 14, 17, 24, 1...","{'labels': [tensor(-100), tensor(0), tensor(52..."
4795,"[24, 16, 16, 16, 7, 16, 27, 16, 17, 27, 24, 12...","{'labels': [tensor(-100), tensor(52), tensor(5..."
4796,"[22, 19, 19, 19, 19, 19, 22, 19, 19, 20, 24, 1...","{'labels': [tensor(-100), tensor(0), tensor(52..."
4797,"[22, 20, 20, 22, 7, 20, 27, 20, 20, 27, 22, 12...","{'labels': [tensor(-100), tensor(52), tensor(5..."
